In [ ]:
!pip install ekphrasis 

In [2]:
#imports 

import pandas as pd
import numpy as np
import json
#from pandas import json_normalize
import re
import string

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.dicts.emoticons import emoticons

#from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report  


%matplotlib inline


In [3]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data preprocessing**
VTPAN Dataset

In [4]:
#read unprocessed train and test data 
path_t = '/content/drive/MyDrive/IM/VTPAN_csv/VTPAN-train.csv'
VTPAN_train = pd.read_csv(path_t)


path_te = '/content/drive/MyDrive/IM/VTPAN_csv/VTPAN-test.csv'
VTPAN_test = pd.read_csv(path_te)

In [5]:
VTPAN_train

,label,chatName,segment
0,non-predator,e621da5de598c9321a1d505ea95e6a2d,Hola. hi. whats up? not a ton. you? same. bein...
1,non-predator,2c8125f8376aa2515c19222ba4213c28,hi hihi m or f?? akjfkjasdkjfajkdfjkakdjflkajd...
2,non-predator,bbf1d909f2a37738a9549ff301144475,Im a 17 F Lesbian. Im looking for girls only j...
3,predator,fc77b9ed8d13697e8deea79de0b4df23,hey!! hi! how are you feeling? a little better...
4,non-predator,b0772b27b7024430b87d4c7ae0a155b9,hi :) hey m or f? f and you? cool m umm im 15 ...
...,...,...,...
6583,non-predator,a09bc8d677c080986c424e7c9953dd5c,69b0d3dfe919a6b860a9fac82de52a7e: if all html ...
6584,non-predator,b29248cde1b661b06108f8fd5b3f9dd3,hello i m 21 male and u 16 female from where? ...
6585,non-predator,874ebdf48ecd8fa35c895b95ebf0e359,cc097e0d7183ae8436e7df709553c8c0: congrats on ...
6586,non-predator,461b64e4568c4be7eb6bbf7a5a32869e,hello m u m or f f how u doin? good im f and u...


In [ ]:
#remove urls 
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

VTPAN_train["clean_text"] = VTPAN_train["segment"].apply(lambda text:  remove_urls(text))
VTPAN_test["clean_text"] = VTPAN_test["segment"].apply(lambda text:  remove_urls(text))

In [ ]:
#remove html tags 
def remove_tags(string):
    result = re.sub('<.*?>','',str(string))
    return result

VTPAN_train["clean_text"]=VTPAN_train["clean_text"].apply(lambda cw : remove_tags(cw))
VTPAN_test["clean_text"] =VTPAN_test["clean_text"] .apply(lambda cw : remove_tags(cw))

In [ ]:
#remove punctuation 
def remove_repeat_punctuation(text):
    """custom function to remove the punctuation"""
    clean = re.sub(r'(?<=[^!.,>$%&-][!.,>$%&-])[!.,>$%& -]+(?<! )', ' ', text)
    return clean
  

VTPAN_train["clean_text"]= VTPAN_train["clean_text"].apply(lambda text: remove_repeat_punctuation(text))
VTPAN_test["clean_text"] = VTPAN_test["clean_text"].apply(lambda text: remove_repeat_punctuation(text))

In [ ]:
# Source : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py

#List of emoticons
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [ ]:
# #remove emoticons 
def remove_emoticons(text):
     emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
     return emoticon_pattern.sub(r'', text)


VTPAN_train["clean_text"] = VTPAN_train["clean_text"] .apply(lambda text: remove_emoticons(text))
VTPAN_test["clean_text"] = VTPAN_test["clean_text"].apply(lambda text: remove_emoticons(text))

In [ ]:
# source: https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt

#list of chat slang
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick :-D  Laugher
"""

In [ ]:
#Chat words conversion

chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

VTPAN_train["clean_text"] = VTPAN_train["clean_text"].apply(lambda text: chat_words_conversion(text))
VTPAN_test["clean_text"] = VTPAN_test["clean_text"].apply(lambda text: chat_words_conversion(text))

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date'],
    # terms that will be annotated
   # annotate={"hashtag", "elongated", "repeated",
        #'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True, # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    #tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


In [ ]:
VTPAN_train["clean_text"] = VTPAN_train["clean_text"] .apply(lambda text: text_processor.pre_process_doc(text))
VTPAN_test["clean_text"] = VTPAN_test["clean_text"] .apply(lambda text: text_processor.pre_process_doc(text))

In [ ]:
#save clean datasets
VTPAN_train.to_csv('VTPAN_train_clean.csv', index=False)
VTPAN_test.to_csv('VTPAN_test_clean.csv', index=False)

In [5]:
#read datasets 
path_train_clean = '/content/drive/MyDrive/IM/VTPAN_csv/VTPAN_train_clean.csv'
VTPAN_train_clean = pd.read_csv(path_train_clean)


path_test_clean = '/content/drive/MyDrive/IM/VTPAN_csv/VTPAN_test_clean.csv'
VTPAN_test_clean = pd.read_csv(path_test_clean)

In [33]:
VTPAN_train_clean

,label,chatName,segment,clean_text
0,non-predator,e621da5de598c9321a1d505ea95e6a2d,Hola. hi. whats up? not a ton. you? same. bein...,Hola. hi. whats up? not a ton. you? same. bein...
1,non-predator,2c8125f8376aa2515c19222ba4213c28,hi hihi m or f?? akjfkjasdkjfajkdfjkakdjflkajd...,hi hihi m or f ? <repeated> akjfkjasdkjfajkdfj...
2,non-predator,bbf1d909f2a37738a9549ff301144475,Im a 17 F Lesbian. Im looking for girls only j...,Im a 17 F Lesbian. Im looking for girls only j...
3,predator,fc77b9ed8d13697e8deea79de0b4df23,hey!! hi! how are you feeling? a little better...,hey! hi! how are you feeling? a little better ...
4,non-predator,b0772b27b7024430b87d4c7ae0a155b9,hi :) hey m or f? f and you? cool m umm im 15 ...,hi hey m or f? f and you? cool m umm im 15 hb...
...,...,...,...,...
6583,non-predator,a09bc8d677c080986c424e7c9953dd5c,69b0d3dfe919a6b860a9fac82de52a7e: if all html ...,69b0d3dfe919a6b860a9fac82de52a7e: if all html ...
6584,non-predator,b29248cde1b661b06108f8fd5b3f9dd3,hello i m 21 male and u 16 female from where? ...,hello i m 21 male and You 16 female from where...
6585,non-predator,874ebdf48ecd8fa35c895b95ebf0e359,cc097e0d7183ae8436e7df709553c8c0: congrats on ...,cc097e0d7183ae8436e7df709553c8c0: congrats on ...
6586,non-predator,461b64e4568c4be7eb6bbf7a5a32869e,hello m u m or f f how u doin? good im f and u...,hello m You m or f f how You doin? good im f a...


In [25]:
#max len of string in train data
len = int(VTPAN_train_clean['clean_text'].str.len().max())
print(len)

40977


In [21]:
#max len of string in test data
len = int(VTPAN_test_clean['clean_text'].str.len().max())
print(len)

410082


In [11]:
#Statistics 
#class distribution, train set 
 
print('In training data:')
print(f'predator:', len(VTPAN_train_clean[VTPAN_train_clean.label=='predator']))

print(f'non-predator:', len(VTPAN_train_clean[VTPAN_train_clean.label=='non-predator']))

In training data:
predator: 798
non-predator: 5790


In [13]:
#Statistics 
#class distribution, test set
print('In test data:') 
print(f'predator:', len(VTPAN_test_clean[VTPAN_test_clean.label=='predator']))
print(f'non-predator:', len(VTPAN_test_clean[VTPAN_test_clean.label=='non-predator']))


In test data:
predator: 1466
non-predator: 13863


In [14]:
#one-hot encoding the labels 
def to_sentiment(label):
  if label == 'predator':
    return 1
  elif label == 'non-predator':
    return 0
 
VTPAN_train_clean['label'] = VTPAN_train_clean['label'].apply(to_sentiment)
VTPAN_test_clean['label'] = VTPAN_test_clean['label'].apply(to_sentiment)

In [15]:
VTPAN_train_clean

,label,chatName,segment,clean_text
0,0,e621da5de598c9321a1d505ea95e6a2d,Hola. hi. whats up? not a ton. you? same. bein...,Hola. hi. whats up? not a ton. you? same. bein...
1,0,2c8125f8376aa2515c19222ba4213c28,hi hihi m or f?? akjfkjasdkjfajkdfjkakdjflkajd...,hi hihi m or f ? <repeated> akjfkjasdkjfajkdfj...
2,0,bbf1d909f2a37738a9549ff301144475,Im a 17 F Lesbian. Im looking for girls only j...,Im a 17 F Lesbian. Im looking for girls only j...
3,1,fc77b9ed8d13697e8deea79de0b4df23,hey!! hi! how are you feeling? a little better...,hey! hi! how are you feeling? a little better ...
4,0,b0772b27b7024430b87d4c7ae0a155b9,hi :) hey m or f? f and you? cool m umm im 15 ...,hi hey m or f? f and you? cool m umm im 15 hb...
...,...,...,...,...
6583,0,a09bc8d677c080986c424e7c9953dd5c,69b0d3dfe919a6b860a9fac82de52a7e: if all html ...,69b0d3dfe919a6b860a9fac82de52a7e: if all html ...
6584,0,b29248cde1b661b06108f8fd5b3f9dd3,hello i m 21 male and u 16 female from where? ...,hello i m 21 male and You 16 female from where...
6585,0,874ebdf48ecd8fa35c895b95ebf0e359,cc097e0d7183ae8436e7df709553c8c0: congrats on ...,cc097e0d7183ae8436e7df709553c8c0: congrats on ...
6586,0,461b64e4568c4be7eb6bbf7a5a32869e,hello m u m or f f how u doin? good im f and u...,hello m You m or f f how You doin? good im f a...
